In [21]:
import pymongo
import pandas as pd
import itertools
import scipy
import pymongo
import argparse
import matplotlib
matplotlib.use('tkagg')

import config
from container import Container
from DataSource import DataSource

import preprocessing_methods1

import warnings
warnings.filterwarnings('ignore')

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['PrenatalTracker']



In [30]:
container = Container()
dataframe = container.prepared_data_provider().get('rec_134_T')

PPG_data = dataframe['PPG'].tolist()
ECG_data = dataframe['ECG'].tolist()
ABP_data = dataframe['BP'].tolist()

In [31]:
import numpy as np
import heartpy as hp
from scipy.signal import find_peaks, argrelextrema
from sklearn.preprocessing import minmax_scale
    
PPG_norm = minmax_scale(PPG_data,feature_range=(0, 1))
    
 # 1st derivative
PPG_d = np.gradient(PPG_norm)
# 2nd derivative
PPG_sd = np.gradient(PPG_d)
    
    # running the PPG peak detection analysis 
wd1, m1 = hp.process(PPG_norm, sample_rate = 125)
wd_d1, m_d1 = hp.process(PPG_d, sample_rate = 125)
wd_sd1, m_sd1 = hp.process(PPG_sd, sample_rate = 125)
    
    # (a) systolic peaks
systolic_peak_idx = wd1['peaklist'] # array of peak indices
systolic_peak = wd1['ybeat'] # array of peak values
    
    # (b1) valley points for PPG
ppg_valley_point_idx = [] # array of valley point indices
ppg_valley_point = [] # array of valley point values
for i in range(len(systolic_peak)-1):
    if i == 0:
        p1 = 0 # beginning of data
        p2 = systolic_peak_idx[i] # index of 1st peak
        ptp = PPG_norm[p1:p2]
        min_val = min(ptp)
        ppg_valley_point.append(min_val)
        min_idx = np.where(ptp == min_val)
        ppg_valley_point_idx.append(min_idx[0][0].tolist()) 
    else: 
        p1 = systolic_peak_idx[i-1] # index of 1st peak
        p2 = systolic_peak_idx[i] # index of 2nd peak
        ptp = PPG_norm[p1:p2]
        min_val = min(ptp)
        ppg_valley_point.append(min_val)
        min_idx = np.where(ptp == min_val) + p1
        ppg_valley_point_idx.append(min_idx[0][0].tolist())
            
    # (b2) valley points and lowest points for dPPG
dppg_peak_idx = wd_d1['peaklist']
dppg_peak = wd_d1['ybeat']
dppg_valley_point_idx = [] # array of valley point indices
dppg_valley_point = [] # array of valley point values
dDA_point_idx = [] # array of lowest point indices
dDA_point = [] # array of lowest point value

    # valley points
for i in range(len(dppg_peak_idx)-1):
    if i == 0:
        p1 = 0 # beginning of data
        p2 = dppg_peak_idx[i] # index of 2nd peak
        ptp = PPG_d[p1:p2]
        mins = argrelextrema(ptp, np.less_equal)
        dppg_valley_point_idx.append(mins[0][-1])
        dppg_valley_point.append(ptp[mins[0][-1]])
    else:
        p1 = dppg_peak_idx[i-1] # index of 1st peak
        p2 = dppg_peak_idx[i] # index of 2nd peak
        ptp = PPG_d[p1:p2]
        if len(ptp) > 20:
            mins = argrelextrema(ptp, np.less_equal)
            if ptp[mins[0][-1]] != 0:
                    dppg_valley_point_idx.append(mins[0][-1] + p1)
                    dppg_valley_point.append(ptp[mins[0][-1]])
            
    # lowest points
for i in range(len(dppg_peak_idx)-1):
    if i == 0:
        p1 = 0 # beginning of data
        p2 = dppg_peak_idx[i] # index of 2nd peak
        ptp = PPG_d[p1:p2]
        min_val = min(ptp)
        dDA_point.append(min_val)
        min_idx = np.where(ptp == min_val)
        dDA_point_idx.append(min_idx[0][0].tolist())
    else:
        p1 = dppg_peak_idx[i-1] # index of 1st peak
        p2 = dppg_peak_idx[i] # index of 2nd peak
        ptp = PPG_d[p1:p2]
        min_val = min(ptp)
        dDA_point.append(min_val)
        min_idx = np.where(ptp == min_val) + p1
        dDA_point_idx.append(min_idx[0][0].tolist())
            
   

In [32]:
ptp[mins[0][-1]]

-0.014842089280291515

In [33]:
mins[0][-1]

58

In [28]:
def process_docs(collection_name):
    
    container = Container()
    dataframe = container.prepared_data_provider().get(collection_name)

    PPG_data = dataframe['PPG'].tolist()
    ECG_data = dataframe['ECG'].tolist()
    ABP_data = dataframe['BP'].tolist()
    
    ECG_features = preprocessing_methods1.ECGPreprocessing(ECG_data)
    PPG_features, PPG_norm = preprocessing_methods1.PPGPreprocessing(PPG_data)
    ABP_features = preprocessing_methods1.BPPreprocessing(ABP_data)

    df_init = preprocessing_methods1.combine_data(ECG_features, PPG_features, ABP_features)
    df = preprocessing_methods1.features(df_init,PPG_norm)
    
    return df


In [29]:
process_docs('rec_134_T')

IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
 # (b3) valley points and lowest points for sdPPG
sdppg_peak_idx = wd_sd1['peaklist']
sdppg_peak = wd_sd1['ybeat']
sdppg_valley_point_idx = [] # array of valley point indices
sdppg_valley_point = [] # array of valley point values
sdDA_point_idx = [] # array of lowest point indices
sdDA_point = [] # array of lowest point value

    # valley points
for i in range(len(sdppg_peak_idx)-1):
    if i == 0:
        p1 = 0 # beginning of data
        p2 = sdppg_peak_idx[i] # index of 2nd peak
        ptp = PPG_sd[p1:p2]
        mins = argrelextrema(ptp, np.less_equal)
        if mins == True:
            sdppg_valley_point_idx.append(mins[0][-1])
            sdppg_valley_point.append(ptp[mins[0][-1]])
        else:
            sdppg_valley_point_idx.append(np.nan)
            sdppg_valley_point.append(np.nan)
    else:
        p1 = sdppg_peak_idx[i-1] # index of 1st peak
        p2 = sdppg_peak_idx[i] # index of 2nd peak
        ptp = PPG_sd[p1:p2]
        if len(ptp) > 20:
            mins = argrelextrema(ptp, np.less_equal)
            if ptp[mins[0][-1]] != 0:
                sdppg_valley_point_idx.append(mins[0][-1] + p1)
                sdppg_valley_point.append(ptp[mins[0][-1]])
            
    # lowest points
for i in range(len(sdppg_peak_idx)-1):
    if i == 0:
        p1 = 0 # beginning of data
        p2 = sdppg_peak_idx[i] # index of 2nd peak
        ptp = PPG_sd[p1:p2]
        min_val = min(ptp)
        sdDA_point.append(min_val)
        min_idx = np.where(ptp == min_val)
        sdDA_point_idx.append(min_idx[0][0].tolist())
    else:
        p1 = sdppg_peak_idx[i-1] # index of 1st peak
        p2 = sdppg_peak_idx[i] # index of 2nd peak
        ptp = PPG_sd[p1:p2]
        min_val = min(ptp)
        sdDA_point.append(min_val)
        min_idx = np.where(ptp == min_val) + p1
        sdDA_point_idx.append(min_idx[0][0].tolist())
            
     # (c) max/min slope values 
slopeval_ppg = [PPG_norm[i] for i in wd_d1['peaklist']]
slopeval_ppg_idx = wd_d1['peaklist']
slopeval_dppg = [PPG_d[i] for i in wd_sd1['peaklist']]
slopeval_dppg_idx = wd_sd1['peaklist']
    
    # (d) dicrotic notch points
dn = find_peaks(PPG_sd, distance=25)
secondary_idx = [dn[0][index] for index in range(0, len(dn[0]), 2)]
dicrotic_notch_idx = [item for item in secondary_idx if PPG_sd[item] <= 0.05]
dicrotic_notch = [PPG_norm[i] for i in dicrotic_notch_idx]
